In [1]:
import numpy as np
import pandas as pd
np.random.seed(1)

In [2]:
dataset = pd.read_csv("winequality-red.csv", sep=",")
dataset.head().style.background_gradient(axis=0)

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.400000,0.700000,0.000000,1.900000,0.076000,11.000000,34.000000,0.997800,3.510000,0.560000,9.400000,5
1,7.800000,0.880000,0.000000,2.600000,0.098000,25.000000,67.000000,0.996800,3.200000,0.680000,9.800000,5
2,7.800000,0.760000,0.040000,2.300000,0.092000,15.000000,54.000000,0.997000,3.260000,0.650000,9.800000,5
3,11.200000,0.280000,0.560000,1.900000,0.075000,17.000000,60.000000,0.998000,3.160000,0.580000,9.800000,6
4,7.400000,0.700000,0.000000,1.900000,0.076000,11.000000,34.000000,0.997800,3.510000,0.560000,9.400000,5


In [3]:
dt_new = dataset.groupby('quality',axis=0).mean().round(2)
dt_new.head().style.background_gradient(axis=0)

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol
quality,,,,,,,,,,,
3,8.360000,0.880000,0.170000,2.640000,0.120000,11.000000,24.900000,1.000000,3.400000,0.570000,9.960000
4,7.780000,0.690000,0.170000,2.690000,0.090000,12.260000,36.250000,1.000000,3.380000,0.600000,10.270000
5,8.170000,0.580000,0.240000,2.530000,0.090000,16.980000,56.510000,1.000000,3.300000,0.620000,9.900000
6,8.350000,0.500000,0.270000,2.480000,0.080000,15.710000,40.870000,1.000000,3.320000,0.680000,10.630000
7,8.870000,0.400000,0.380000,2.720000,0.080000,14.050000,35.020000,1.000000,3.290000,0.740000,11.470000


In [4]:
c = []
for q in dataset['quality'].values:
    if q < 6:
        c.append([1,0,0])
    elif q > 6:
        c.append([0,0,1])
    else:
        c.append([0,1,0])
ca = np.array(c)
ma = np.arange(1599)

In [5]:
X = dataset.drop(["quality","density"],axis=1)

In [6]:
ones = np.ones((1,X.shape[0]))
X = np.hstack((ones.T,X))

In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, N_train, N_test = train_test_split(X, ma, train_size=0.8, random_state=10000)

In [8]:
def convert_label(c , N, size):
    y=[]
    for i in range(size):
       y.append(c[N[i]])
    return y
def test_samples(ca, N, size):
    y=[]
    for i in range(size):
        if ca[N[i]] == [1,0,0]:
            y.append(0)
        if ca[N[i]] == [0,0,1]:
            y.append(2)
        if ca[N[i]] == [0,1,0]:
            y.append(1)
    return y
y_train = np.array(convert_label(c,N_train,size=1279))
y_test = np.array(test_samples(c,N_test,size=320))
y_test

array([1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 2, 0, 1, 1, 1, 0, 0,
       0, 1, 1, 1, 0, 0, 2, 0, 0, 2, 0, 0, 1, 0, 0, 0, 0, 0, 2, 1, 1, 1,
       1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 2, 0, 1, 1,
       0, 1, 0, 2, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 2, 0, 0, 1, 0, 0, 1, 0,
       2, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0,
       1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1,
       1, 2, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 2, 0, 1, 2, 0, 0, 1, 0, 1, 2,
       1, 0, 0, 1, 2, 1, 0, 1, 1, 0, 0, 0, 1, 0, 2, 2, 2, 1, 0, 0, 1, 1,
       0, 0, 1, 1, 0, 0, 1, 0, 1, 2, 1, 1, 1, 2, 1, 2, 1, 2, 0, 1, 0, 2,
       1, 0, 0, 1, 2, 0, 0, 0, 0, 1, 0, 1, 2, 1, 0, 1, 2, 0, 0, 0, 0, 0,
       1, 1, 0, 2, 1, 2, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 2, 1, 0, 1, 1, 1, 0,
       1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 2, 1, 0, 0, 0, 1, 0, 1, 1,
       1, 1, 0, 0, 1, 0, 2, 1, 1, 0, 1, 0, 0, 1, 0,

In [9]:
W_init = np.random.rand(X_train.shape[1],3)
y_train = y_train.T
X_train = X_train.T

In [10]:
def softmax(Z):
    """
    X ∈ R(d,N) ,
    y ∈ R(c,N) ,
    W ∈ R(d,c) ,
    :param Z: = W.T.dot(X) ∈ R(c,N) .
    :return: softmax(z)
    """
    e_Z = np.exp(Z -np.max(Z, axis=0,keepdims=True))
    return e_Z / e_Z.sum(axis=0)

In [11]:
def cost(X, y, W):
    """
    :param X: ∈ R(d,N)
    :param y: ∈ R(c,N)
    :param W: W ∈ R(d,c)
    :return: cross entropy
    """
    a = softmax(W.T.dot(X))
    return - np.sum(y*np.log(a))

In [12]:
def gradient(X, y, W):
    a = softmax(W.T.dot(X))
    e = a - y
    return X.dot(e.T)

In [13]:
def update(X, y, W_init , epouch, eta, tol=1e-4):

    """
    :param X: ∈ R(d,N)
    :param y: ∈ R(c,N)
    :param W_init: W ∈ R(d,c)
    """

    W = [W_init]
    C = W_init.shape[1]
    N = X.shape[1]
    d = X.shape[0]
    count =0
    check_w_after = 20
    while count<epouch:
        mix_id = np.random.permutation(N)
        for i in mix_id:
            xi = X[:, i].reshape(d, 1)
            yi = y[:, i].reshape(C, 1)
            ai = softmax(np.dot(W[-1].T,xi))
            W_new  = W[-1] + eta * xi . dot((yi-ai).T) / N
            count+= 1
            if count%check_w_after==0:
                if np.linalg.norm(W_new -  W[-check_w_after])<tol:
                    return W
            W.append(W_new)
    return W

In [14]:
eta = .05
epouch = 1000000
W = update(X_train,y_train,W_init,epouch,eta)
W[-1]

array([[ 0.73277362,  0.65368389, -0.24899664],
       [ 0.25404776,  0.18557747,  0.10180183],
       [ 1.03474689,  0.0301309 , -0.13628937],
       [ 0.18730369,  0.46587225,  0.99005481],
       [ 0.3530641 ,  0.35814728,  0.39874589],
       [ 0.76231095,  0.39376622,  0.49038496],
       [ 0.36049516,  0.39108364,  0.38765419],
       [ 0.67554029,  0.65584262,  0.64262546],
       [ 2.01338489,  0.66766998, -0.82501484],
       [-0.30673715,  0.36170452,  1.03206034],
       [-0.02344306,  0.51081497,  0.98997208]])

In [15]:
def pred(X, W):
    A = softmax((W.T).dot(X))
    return np.argmax(A,axis=0)

In [16]:
y_res = pred(X_test.T, W[-1])

In [17]:
from sklearn.metrics import accuracy_score
print("Accuracy score: {} %.".format(accuracy_score(y_test,y_res)*100))

Accuracy score: 62.5 %.
